# Cleaning Data with Pandas
## Reading the data

In [277]:
import numpy as np
import pandas as pd

from pymongo import MongoClient

def get_mongo_database(db_name, host='localhost', port=27017, username=None, password=None):
    '''Get (or create) named database from MongoDB with/out authentication'''
    if username and password:
        mongo_uri = 'mongodb://{}:{}@{}/{}'.format(username, password, host, db_name)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)
    return conn[db_name]

def mongo_to_dataframe(db_name, collection_name, query={}, host='localhost',
                       port=27017, username=None, password=None, no_id=True):
    '''Create a Pandas DataFrame from MongoDB collection'''
    db = get_mongo_database(db_name, host, port, username, password)
    cursor = db[collection_name].find(query)
    df = pd.DataFrame(list(cursor))
    if no_id:
        del df['_id']
    return df

def dataframe_to_mongo(dframe, db_name, collection_name, host='localhost',
                 port=27017, username=None, password=None):
    '''save a dataframe to mongodb collection'''
    db = get_mongo_database(db_name, host, port, username, password)
    records = dframe.to_dict('records')  # 'records' puts it into our list-of-dicts format
    db[collection_name].insert_many(records)


DB_NOBEL_PRIZE = 'nobel_prize' # use string constants or a spell error in retrieval will create new table.
COLL_WINNERS = 'winners' # winners collection

#----------------------------
# From json file.
#----------------------------

with open('data/nwinners.json') as f:
    df = pd.read_json(f)

# Make sure mongodb is clear (so we don't duplicate data), then
# save to Mongo for next section
db = get_mongo_database(DB_NOBEL_PRIZE)
db[COLL_WINNERS].delete_many({})  # deletes everything (no filter)
dataframe_to_mongo(df, DB_NOBEL_PRIZE, COLL_WINNERS) # save to Mongo for next section

#----------------------------
# From mongodb collection
#----------------------------

df = mongo_to_dataframe(DB_NOBEL_PRIZE, COLL_WINNERS)


## Inspecting the data

In [278]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1069 entries, 0 to 1068
Data columns (total 12 columns):
born_in           1069 non-null object
category          1069 non-null object
country           1069 non-null object
date_of_birth     1060 non-null object
date_of_death     712 non-null object
gender            1060 non-null object
link              1069 non-null object
name              1069 non-null object
place_of_birth    1060 non-null object
place_of_death    712 non-null object
text              1069 non-null object
year              1069 non-null int64
dtypes: int64(1), object(11)
memory usage: 100.3+ KB


In [279]:
df.describe() # only acts on numeric columns
df.describe(include=['object']) # include specifies other types to include (besides numeric)

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text
count,1069,1069,1069,1060,712,1060,1069,1069,1060,712,1069
unique,33,7,59,868,589,2,912,1002,607,316,1057
top,,Physiology or Medicine,United States,7 November 1867,4 July 1934,male,https://en.wikipedia.org/wiki/Marie_Curie,César Milstein,New York City,Cambridge,"John Harsanyi , Economics, 1994"
freq,938,256,352,4,4,1003,4,3,44,37,2


In [280]:
df.tail()
df.head(3)

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year
0,,Physiology or Medicine,Summary,8 October 1927,24 March 2002,male,https://en.wikipedia.org/wiki/C%C3%A9sar_Milstein,César Milstein,Bahía Blanca,Cambridge,"César Milstein , Physiology or Medicine, 1984",1984
1,Austria,Physiology or Medicine,,7 November 1929,NaN,male,https://en.wikipedia.org/wiki/Eric_Kandel,Eric Kandel *,Vienna,NaN,"Eric Kandel *, Physiology or Medicine, 2000",2000
2,Austria,Chemistry,,9 March 1923,19 April 2016,male,https://en.wikipedia.org/wiki/Walter_Kohn,Walter Kohn *,Vienna,Santa Barbara,"Walter Kohn *, Chemistry, 1998",1998


## Set indices (optional, but useful)

In [281]:
print(df.columns) # the column index
df = df.set_index('name') # row index (note we need to assign to df)
df.loc['Albert Einstein'] # loc accesses by label index

Index([u'born_in', u'category', u'country', u'date_of_birth', u'date_of_death',
       u'gender', u'link', u'name', u'place_of_birth', u'place_of_death',
       u'text', u'year'],
      dtype='object')


,born_in,category,country,date_of_birth,date_of_death,gender,link,place_of_birth,place_of_death,text,year
name,,,,,,,,,,,
Albert Einstein,,Physics,Switzerland,14 March 1879,18 April 1955,male,https://en.wikipedia.org/wiki/Albert_Einstein,Ulm,Princeton,"Albert Einstein , born in Germany , Physics, ...",1921
Albert Einstein,,Physics,Germany,14 March 1879,18 April 1955,male,https://en.wikipedia.org/wiki/Albert_Einstein,Ulm,Princeton,"Albert Einstein , Physics, 1921",1921


In [282]:
df.reset_index(inplace=True) # go back to default int index.  Note the 'inplace'
df.iloc[2] # iloc accesses by integer index
df.head(2)

,name,born_in,category,country,date_of_birth,date_of_death,gender,link,place_of_birth,place_of_death,text,year
0,César Milstein,,Physiology or Medicine,Summary,8 October 1927,24 March 2002,male,https://en.wikipedia.org/wiki/C%C3%A9sar_Milstein,Bahía Blanca,Cambridge,"César Milstein , Physiology or Medicine, 1984",1984
1,Eric Kandel *,Austria,Physiology or Medicine,,7 November 1929,NaN,male,https://en.wikipedia.org/wiki/Eric_Kandel,Vienna,NaN,"Eric Kandel *, Physiology or Medicine, 2000",2000


In [283]:
df[0:10] #first 10 rows
df[-4:]  #last 4 rows

# masks
mask = df.year > 2000
df[mask]
df[df.year>2000] # more direct

,name,born_in,category,country,date_of_birth,date_of_death,gender,link,place_of_birth,place_of_death,text,year
3,Elfriede Jelinek,,Literature,Austria,20 October 1946,NaN,female,https://en.wikipedia.org/wiki/Elfriede_Jelinek,Mürzzuschlag,NaN,"Elfriede Jelinek , Literature, 2004",2004
12,Brian Schmidt,,Physics,Australia,24 February 1967,NaN,male,https://en.wikipedia.org/wiki/Brian_Schmidt,Missoula,NaN,"Brian Schmidt , born in the United States , P...",2011
13,Elizabeth Blackburn *,Australia,Physiology or Medicine,,26 November 1948,NaN,female,https://en.wikipedia.org/wiki/Elizabeth_Blackburn,Hobart,NaN,"Elizabeth Blackburn *, Physiology or Medicine,...",2009
14,Barry Marshall,,Physiology or Medicine,Australia,30 September 1951,NaN,male,https://en.wikipedia.org/wiki/Barry_Marshall,Kalgoorlie,NaN,"Barry Marshall , Physiology or Medicine, 2005",2005
15,Harald zur Hausen,,Physiology or Medicine,Germany,11 March 1936,NaN,male,https://en.wikipedia.org/wiki/Harald_zur_Hausen,Gelsenkirchen,NaN,"Harald zur Hausen , Physiology or Medicine, 2008",2008
16,Gerhard Ertl,,Chemistry,Germany,10 October 1936,NaN,male,https://en.wikipedia.org/wiki/Gerhard_Ertl,Stuttgart,NaN,"Gerhard Ertl , Chemistry, 2007",2007
31,David J. Gross,,Physics,United States,19 February 1941,NaN,male,https://en.wikipedia.org/wiki/David_J._Gross,"Washington, D.C.",NaN,"David J. Gross , Physics, 2004",2004
54,Tawakkol Karman,,Peace,Yemen,7 February 1979,NaN,female,https://en.wikipedia.org/wiki/Tawakkol_Karman,Ta'izz,NaN,"Tawakkol Karman , Peace, 2011",2011
221,Irwin Rose,,Chemistry,United States,16 July 1926,2 June 2015,male,https://en.wikipedia.org/wiki/Irwin_Rose,Brooklyn,Deerfield,"Irwin Rose , Chemistry, 2004",2004
222,Edward C. Prescott,,Economics,United States,26 December 1940,NaN,male,https://en.wikipedia.org/wiki/Edward_C._Prescott,Glens Falls,NaN,"Edward C. Prescott , Economics, 2004",2004


## Cleaning the Data
### Remove the asterisk from names

People that got the award in a different country than they were born in were entered two times in the original wikipedia page where we scraped this data.  An example is John Cornforth who won the award in the UK, but was born in Australia.  Look at his records:

In [284]:
df[df.name.str.contains('Cornforth')]

,name,born_in,category,country,date_of_birth,date_of_death,gender,link,place_of_birth,place_of_death,text,year
384,John Cornforth,,Chemistry,United Kingdom,7 September 1917,8 December 2013,male,https://en.wikipedia.org/wiki/John_Cornforth,Sydney,Brighton,"John Cornforth , born in Australia , Chemistr...",1975
837,John Cornforth *,Australia,Chemistry,,7 September 1917,8 December 2013,male,https://en.wikipedia.org/wiki/John_Cornforth,Sydney,Brighton,"John Cornforth *, Chemistry, 1975",1975


In [285]:
# How many names with asterisk?
df[df.name.str.contains('\*')]['name'].count()

131

The entry that had an asterisk by the name was an entry in the country the person was born in as opposed to the country from which they received the award.  In these cases, we need to get rid of the duplicate entry but also enter the born_in field in the original entry without the asterisk.

In [286]:
# Take out the asterisks in people's name
df.name = df.name.str.replace('*', '')
df.name = df.name.str.strip() # strip any remaining whitespace at end
df[df.name.str.contains('\*')]['name'].count()

0

### Eliminating duplications for people with a born_in entry

In [287]:
set(df.born_in.apply(type))

{unicode}

In [288]:
bornin_col = df.born_in # or df['born_in']

# if you look at bornin_col, most entries are an empty string
# replace empty strings with NaN
bornin_col.replace('', np.nan, inplace=True)
bornin_coln

NameError: name 'bornin_coln' is not defined

In [ ]:
# now do that NaN replacement for all '' entries in the dataframe
df.replace('',np.nan, inplace=True)

### DANNY - new section to transfer born_in entries into the duplicate row of same name which has a country entry but no born_in entry

In [289]:
df[df.name.str.contains('Cornforth')]

,name,born_in,category,country,date_of_birth,date_of_death,gender,link,place_of_birth,place_of_death,text,year
384,John Cornforth,NaN,Chemistry,United Kingdom,7 September 1917,8 December 2013,male,https://en.wikipedia.org/wiki/John_Cornforth,Sydney,Brighton,"John Cornforth , born in Australia , Chemistr...",1975
837,John Cornforth,Australia,Chemistry,,7 September 1917,8 December 2013,male,https://en.wikipedia.org/wiki/John_Cornforth,Sydney,Brighton,"John Cornforth *, Chemistry, 1975",1975


In [290]:
df[df.name.str.contains('Leonid Hurwicz')]

,name,born_in,category,country,date_of_birth,date_of_death,gender,link,place_of_birth,place_of_death,text,year
352,Leonid Hurwicz,NaN,Economics,United States,21 August 1917,24 June 2008,male,https://en.wikipedia.org/wiki/Leonid_Hurwicz,Moscow,Minneapolis,"Leonid Hurwicz , born in Russia , Economics, ...",2007
567,Leonid Hurwicz,Russia and Soviet Union,Economics,,21 August 1917,24 June 2008,male,https://en.wikipedia.org/wiki/Leonid_Hurwicz,Moscow,Minneapolis,"Leonid Hurwicz *, Economics, 2007",2007
666,Leonid Hurwicz,Poland,Economics,,21 August 1917,24 June 2008,male,https://en.wikipedia.org/wiki/Leonid_Hurwicz,Moscow,Minneapolis,"Leonid Hurwicz *, born in then Russian Republ...",2007


In [253]:
# an example of join from a stackoverflow answer
# https://stackoverflow.com/questions/39816671/pandas-populate-new-dataframe-column-based-on-matching-columns-in-another-datafr
example_df = pd.DataFrame(dict(
        AUTHOR_NAME=list('AAABBCCCCDEEFGG'),
        title=      list('zyxwvutsrqponml')
    ))

example_df2 = pd.DataFrame(dict(
        AUTHOR_NAME=list('AABCCEGG'),
        title      =list('zwvtrpml'),
        CATEGORY   =list('11223344')
    ))
cols = ['AUTHOR_NAME', 'title']
example_df.join(example_df2.set_index(cols), on=cols, how='left')

,AUTHOR_NAME,title,CATEGORY
0,A,z,1
1,A,y,NaN
2,A,x,NaN
3,B,w,NaN
4,B,v,2
5,C,u,NaN
6,C,t,2
7,C,s,NaN
8,C,r,3
9,D,q,NaN


In [254]:
df_born_in = df[df.born_in.notnull()] # save off the entries with born_in
df_test = df[df.born_in.isnull()] # delete the rows with born_in (by saving only isnull())
df_test = df_test.drop('born_in', axis=1)
match_cols = ['name', 'year', 'category']
df_born_in = df_born_in.set_index(match_cols).loc[:,'born_in']
df_born_in

name                        year  category              
Eric Kandel                 2000  Physiology or Medicine                    Austria
Walter Kohn                 1998  Chemistry                                 Austria
Elizabeth Blackburn         2009  Physiology or Medicine                  Australia
T. S. Eliot                 1948  Literature                          United States
Czesław Miłosz              1980  Literature                          United States
Har Gobind Khorana          1968  Physiology or Medicine              United States
Ben R. Mottelson            1975  Physics                             United States
Subrahmanyan Chandrasekhar  1983  Physics                             United States
Daniel Kahneman             2002  Economics                           United States
Robert Aumann               2005  Economics                           United States
Venkatraman Ramakrishnan    2009  Chemistry                           United States
Christian de Duve  

In [255]:
df_born_in = df[df.born_in.notnull()] # save off the entries with born_in
df = df[df.born_in.isnull()] # delete the rows with born_in (by saving only isnull())
df = df.drop('born_in', axis=1) #drop the born_in column
# note, if you didn't drop the born_in column in df, then the join below
# would have overlapping columns.  With overlapping columns, you must supply
# either an rsuffix or lsuffix so it can specify which of the overlapping columns
# came from which side.  To avoid that, just drop the overlapping column in df and
# specify only the born_in column in df_born_in inside the join statement.

match_cols = ['name', 'year', 'category']
newdf = df.join(df_born_in.set_index(match_cols).loc[:,'born_in'], on=match_cols, sort=True)


In [259]:
newdf[newdf.name.str.contains('Cornforth')]

,name,category,country,date_of_birth,date_of_death,gender,link,place_of_birth,place_of_death,text,year,born_in
384,John Cornforth,Chemistry,United Kingdom,7 September 1917,8 December 2013,male,https://en.wikipedia.org/wiki/John_Cornforth,Sydney,Brighton,"John Cornforth , born in Australia , Chemistr...",1975,Australia


In [257]:
newdf.head()

,name,category,country,date_of_birth,date_of_death,gender,link,place_of_birth,place_of_death,text,year,born_in
296,A. Michael Spence,Economics,United States,7 November 1943,NaN,male,https://en.wikipedia.org/wiki/A._Michael_Spence,"Montclair, New Jersey",NaN,"A. Michael Spence , Economics, 2001",2001,NaN
986,Aage Bohr,Physics,Denmark,19 June 1922,8 September 2009,male,https://en.wikipedia.org/wiki/Aage_Bohr,Copenhagen,Copenhagen,"Aage Bohr , Physics, 1975",1975,NaN
766,Aaron Ciechanover,Chemistry,Israel,1 October 1947,NaN,male,https://en.wikipedia.org/wiki/Aaron_Ciechanover,Haifa,NaN,"Aaron Ciechanover , Chemistry, 2004",2004,NaN
504,Aaron Klug,Chemistry,United Kingdom,11 August 1926,NaN,male,https://en.wikipedia.org/wiki/Aaron_Klug,Želva,NaN,"Aaron Klug , born in Lithuania , Chemistry, 1982",1982,Lithuania
518,Aaron Klug,Chemistry,South Africa,11 August 1926,NaN,male,https://en.wikipedia.org/wiki/Aaron_Klug,Želva,NaN,"Aaron Klug , Chemistry, 1982",1982,Lithuania


## Back to book flow

In [12]:
# Rows with born_in are duplicate entries, so remove all rows
# which are NaN in bornin_col
df = df[df.born_in.isnull()]
print(df.count()) # note that born_in count is now 0
df = df.drop('born_in', axis=1) # drop the born_in column

name              938
born_in             0
category          938
country           938
date_of_birth     929
date_of_death     627
gender            929
link              938
place_of_birth    929
place_of_death    627
text              938
year              938
dtype: int64


### Finding other duplicates
Internet search shows that 889 people and organizations have received Nobel prize, but our data set contains more, so let's look for more duplications

In [44]:
dupes_by_name = df[df.duplicated('name')]
dupes_by_name.count()

category          4
country           4
date_of_birth     4
date_of_death     4
gender            4
link              4
name              4
place_of_birth    4
place_of_death    4
text              4
year              4
award_age         4
bio_image         4
image_urls        4
mini_bio          4
dtype: int64

In [45]:
# By default, 'duplicated' gives the first occurance of a duplicated item
# but it has an option 'keep="last"'.  Using both options, we can get a full
# list of all the duplicated items
all_dupes = df[df.duplicated('name') | df.duplicated('name', keep='last')]
all_dupes.count()

# we could have also done this in the following way
# using the 'isin' method
all_dupes = df[df.name.isin(dupes_by_name.name)]
all_dupes.count()

# we could have also done this in the following way
# groupby returns iterator of (groupname, dataframeofrows) tuples, e.g.
for name, rows in df.groupby('name'):
    if len(rows)>1:
        print('name: {}, number of rows: {}'.format(name.encode('utf-8'), len(rows)))
# use groupby as follows:
pd.concat([g[['name','category','country','year']] for _, g in df.groupby('name') if len(g) > 1])

name: Frederick Sanger, number of rows: 2
name: John Bardeen, number of rows: 2
name: Linus C. Pauling, number of rows: 2
name: Marie Skłodowska-Curie, number of rows: 2


,name,category,country,year
734,Frederick Sanger,Chemistry,United Kingdom,1958.0
736,Frederick Sanger,Chemistry,United Kingdom,1980.0
332,John Bardeen,Physics,United States,1956.0
334,John Bardeen,Physics,United States,1972.0
343,Linus C. Pauling,Chemistry,United States,1954.0
345,Linus C. Pauling,Peace,United States,1962.0
887,Marie Skłodowska-Curie,Physics,Poland,1903.0
891,Marie Skłodowska-Curie,Chemistry,France,1911.0


In [46]:
# Sort the data now
# here is how sort_values works:
df2 = pd.DataFrame(\
                  {'name':['zak', 'alice', 'bob', 'mike',
                          'bob', 'bob'],
                  'score':[4, 3, 5, 2, 3, 7]})
df2 = df2.sort_values(['name', 'score'], ascending=[1,0])
print(df2)
# Now use it on all_dupes
all_dupes.sort_values('name')[['name', 'country', 'year']]

    name  score
1  alice      3
5    bob      7
2    bob      5
4    bob      3
3   mike      2
0    zak      4


,name,country,year
734,Frederick Sanger,United Kingdom,1958.0
736,Frederick Sanger,United Kingdom,1980.0
332,John Bardeen,United States,1956.0
334,John Bardeen,United States,1972.0
343,Linus C. Pauling,United States,1954.0
345,Linus C. Pauling,United States,1962.0
887,Marie Skłodowska-Curie,Poland,1903.0
891,Marie Skłodowska-Curie,France,1911.0


In [16]:
# This output shows that some recipients are in our dataset
# twice for the same year, but different countries.

# Change Curie's country to France for the 1911 entry
df.loc[(df.name == u'Marie Sk\u0142odowska-Curie') &\
      (df.year == 1911), 'country'] = 'France'

df.drop(df[(df.name=='Sidney Altman') & (df.year==1990)].index, inplace=True)
# could have done this the following way as well:
df = df[~((df.name=='Sidney Altman') & (df.year==1990))] # ~ is the logical 'not'


In [17]:
# define function for doing these cleanup activities
def clean_data(df):
    df = df.replace('', np.nan)
    df = df[df.born_in.isnull()] # eliminates the rows with 'born_in' values
    df = df.drop('born_in', axis=1) # don't need this column anymore
    df.drop(df[df.year==1809].index, inplace=True)
    df = df[~(df['name']=='Marie Curie')]
    df.loc[(df.name == u'Marie Sk\u0142odowska-Curie') &\
      (df.year == 1911), 'country'] = 'France'
    df = df[~((df.name=='Sidney Altman') & (df.year==1990))] # selects all rows 'not' (~) Sidney Altma 1990
    return df

df = clean_data(df)

In [18]:
# get rid of half of the dual country winners so each prize just counts once.
# to be fair, randomize the rows first so that country deleted is random.
df = df.reindex(np.random.permutation(df.index)) # randomize the index
# drop duplicate rows (both 'name' and 'year' the same) after the first encountered
df = df.drop_duplicates(['name', 'year'])
df = df.sort_index() # go back to unrandomized index
df.count()

name              883
category          879
country           883
date_of_birth     875
date_of_death     593
gender            875
link              883
place_of_birth    875
place_of_death    593
text              883
year              883
dtype: int64

In [47]:
# lest see how many duplicate winners we have now (some are valid)
df[df.duplicated('name') | df.duplicated('name', keep='last')]\
    .sort_values(by='name')[['name', 'country', 'year', 'category']]
    
# The output are the 4 valid multi-award winners!

,name,country,year,category
734,Frederick Sanger,United Kingdom,1958.0,Chemistry
736,Frederick Sanger,United Kingdom,1980.0,Chemistry
332,John Bardeen,United States,1956.0,Physics
334,John Bardeen,United States,1972.0,Physics
343,Linus C. Pauling,United States,1954.0,Chemistry
345,Linus C. Pauling,United States,1962.0,Peace
887,Marie Skłodowska-Curie,Poland,1903.0,Physics
891,Marie Skłodowska-Curie,France,1911.0,Chemistry


In [48]:
df[df.year.isin([2016, 2017])] #my total differs from book, so look at recent winners...

,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year,award_age,bio_image,image_urls,mini_bio
162,Economics,Finland,1949-04-18,NaT,male,https://en.wikipedia.org/wiki/Bengt_R._Holmstr...,Bengt R. Holmström,Helsinki,NaN,"Bengt R. Holmström , Economics, 2016",2016.0,67.0,full/071002a0805cc5f8c9547c7ced7754740d864163.jpg,[https://upload.wikimedia.org/wikipedia/common...,<p><b>Bengt Robert Holmström</b> (born 18 Apri...
164,Chemistry,France,1944-10-21,NaT,male,https://en.wikipedia.org/wiki/Jean-Pierre_Sauvage,Jean-Pierre Sauvage,Paris,NaN,"Jean-Pierre Sauvage , Chemistry, 2016",2016.0,72.0,full/fe9720dd654cdad9f65e70738e78c765de4366ba.jpg,[https://upload.wikimedia.org/wikipedia/common...,<p><b>Jean-Pierre Sauvage</b> (<small>French p...
192,Peace,Colombia,1951-08-10,NaT,male,https://en.wikipedia.org/wiki/Juan_Manuel_Santos,Juan Manuel Santos,Bogota,NaN,"Juan Manuel Santos , Peace, 2016",2016.0,65.0,full/eff1c4010469810f2dfa814711106255060c618b.jpg,[https://upload.wikimedia.org/wikipedia/common...,<p><b>Juan Manuel Santos Calderón</b> (<small>...
533,Literature,United States,1941-05-24,NaT,male,https://en.wikipedia.org/wiki/Bob_Dylan,Bob Dylan,Duluth,NaN,"Bob Dylan , Literature, 2016",2016.0,75.0,full/4cb79033375bc66bf1ec295e6c1633985665c747.jpg,[https://upload.wikimedia.org/wikipedia/common...,"<p><b>Bob Dylan</b> (<span class=""nowrap""><spa..."
667,Chemistry,United States,1942-05-24,NaT,male,https://en.wikipedia.org/wiki/Fraser_Stoddart,Fraser Stoddart,Edinburgh,NaN,"Fraser Stoddart , born in United Kingdom , Ch...",2016.0,74.0,full/0455003b3d009c6cb9e33dabb746a876e030647c.jpg,[https://upload.wikimedia.org/wikipedia/common...,<p><b>Sir James Fraser Stoddart</b> <span styl...
669,Physics,United States,1951-09-14,NaT,male,https://en.wikipedia.org/wiki/F._Duncan_M._Hal...,F. Duncan M. Haldane,London,NaN,"F. Duncan M. Haldane , born in United Kingdom...",2016.0,65.0,full/f5166c0ccafcc56c3b61ca5f2ce361a97a9658ad.jpg,[https://upload.wikimedia.org/wikipedia/common...,<p><b>Frederick Duncan Michael Haldane</b> <sp...
671,Physics,United States,1943-06-22,NaT,male,https://en.wikipedia.org/wiki/John_M._Kosterlitz,John M. Kosterlitz,Aberdeen,NaN,"John M. Kosterlitz , born in United Kingdom ,...",2016.0,73.0,full/0caf9aac08352aa229bf8d8a33bb28e8336fcfa7.jpg,[https://upload.wikimedia.org/wikipedia/common...,<p><b>John Michael Kosterlitz</b> (born June 2...
799,Economics,United Kingdom,1948-10-09,NaT,male,https://en.wikipedia.org/wiki/Oliver_Hart_(eco...,Oliver Hart,London,NaN,"Oliver Hart , Economics, 2016",2016.0,68.0,full/29d0c0c75945eada060987640bc150122876e151.jpg,[https://upload.wikimedia.org/wikipedia/common...,<p><b>Oliver Simon D'Arcy Hart</b> (born on Oc...
801,Physics,United Kingdom,1934-09-21,NaT,male,https://en.wikipedia.org/wiki/David_J._Thouless,David J. Thouless,Bearsden,NaN,"David J. Thouless , Physics, 2016",2016.0,82.0,full/76dab6a284b4513e70375a7aae257743eccfe946.jpg,[https://upload.wikimedia.org/wikipedia/common...,"<p><b>David James Thouless</b> <span style=""fo..."
953,Chemistry,Netherlands,1951-05-18,NaT,male,https://en.wikipedia.org/wiki/Ben_Feringa,Ben Feringa,Barger-Compascuum,NaN,"Ben Feringa , Chemistry, 2016",2016.0,65.0,full/2334592ca03c102b4d1acf82483f5d6bf3c327b9.jpg,[https://upload.wikimedia.org/wikipedia/common...,"<p><b>Bernard Lucas ""Ben"" Feringa</b> (<small>..."


## Dealing with missing fields
First: missing categories

In [49]:
print(df.count())  # we are missing some categories and genders
df.category.unique() # recall when we scraped, we checked against this limited list.

category          873
country           873
date_of_birth     873
date_of_death     592
gender            873
link              873
name              873
place_of_birth    873
place_of_death    592
text              873
year              873
award_age         873
bio_image         824
image_urls        873
mini_bio          873
dtype: int64


array([u'Chemistry', u'Physiology or Medicine', u'Peace', u'Literature',
       u'Physics', u'Economics'], dtype=object)

In [50]:
# look at the missing categories
df[df.category.isnull()][['name', 'text', 'link', 'category']]

,name,text,link,category


In [23]:
df.loc[df.name == u'Leopold Ružička born in Kingdom of Hungary', ['name', 'text', 'category']] # doesn't work

,name,text,category


In [24]:
print(u'Leopold Ružička born in Kingdom of Hungary'.encode('ascii', errors='backslashreplace'))
print(u'Róbert Bárány'.encode('ascii', errors='backslashreplace'))
df.loc[df.name==u'Leopold Ru\u017ei\u010dka born in Kingdom of Hungary', ['name', 'text', 'category']] # still doesn't work

Leopold Ru\u017ei\u010dka born in Kingdom of Hungary
R\xf3bert B\xe1r\xe1ny


,name,text,category


In [25]:
df.loc[df.name.str.startswith('Leopold Ru'), :] # works!

,name,category,country,date_of_birth,date_of_death,gender,link,place_of_birth,place_of_death,text,year
9,Leopold Ružička born in Kingdom of Hungary,NaN,Hungary,13 September 1887,26 September 1976,male,https://en.wikipedia.org/wiki/Leopold_Ru%C5%BE...,Vukovar,Mammern,Leopold Ružička born in Kingdom of Hungary,0
834,Leopold Ružička,Chemistry,Switzerland,13 September 1887,26 September 1976,male,https://en.wikipedia.org/wiki/Leopold_Ru%C5%BE...,Vukovar,Mammern,"Leopold Ružička , born in then Austria-Hungar...",1939


In [26]:
df.loc[df.name.str.startswith(u'R\xf3bert B\xe1r\xe1ny')]

,name,category,country,date_of_birth,date_of_death,gender,link,place_of_birth,place_of_death,text,year
263,Róbert Bárány,Physiology or Medicine,Austria,22 April 1876,8 April 1936,male,https://en.wikipedia.org/wiki/R%C3%B3bert_B%C3...,Vienna,Uppsala domkyrkoförsamling,"Róbert Bárány , Physiology or Medicine, 1914",1914


In [27]:
def clean_data(df):
    df = df.replace('', np.nan)
    if 'born_in' in df.columns:
        df = df[df.born_in.isnull()] # eliminates the rows with 'born_in' values
        df = df.drop('born_in', axis=1) # don't need this column anymore
    df.drop(df[df.year==1809].index, inplace=True)
    df = df[~(df['name']=='Marie Curie')]
    df.loc[(df.name == u'Marie Sk\u0142odowska-Curie') &\
      (df.year == 1911), 'country'] = 'France'
    df = df[~((df.name=='Sidney Altman') & (df.year==1990))] # selects all rows 'not' (~) Sidney Altma 1990
    # the missing categories are mostly people whose category name wasn't in standard format
    # fix that:
    df.loc[df.name=='Alexis Carrel', 'category'] = 'Physiology or Medicine'
    df.loc[df.name=='Ilya Ilyich Mechnikov', 'category'] = 'Physiology or Medicine'
    df.loc[df.name=='Amartya Sen', 'category'] = 'Economics'
    df.drop(df.loc[df.name.str.startswith('Leopold Ru') & (df.year != 1939)].index, inplace=True)
    df.loc[df.name.str.startswith(u'R\xf3bert B\xe1r\xe1ny'), 'category'] = 'Physiology or Medicine'
    return df

df = clean_data(df)

Now, missing gender

In [51]:
df[df['gender'].isnull()][['name', 'year', 'category']]

,name,year,category


In [29]:
def clean_data(df):
    df = df.replace('', np.nan)
    if 'born_in' in df.columns:
        df = df[df.born_in.isnull()] # eliminates the rows with 'born_in' values
        df = df.drop('born_in', axis=1) # don't need this column anymore
    df.drop(df[df.year==1809].index, inplace=True)
    df = df[~(df['name']=='Marie Curie')]
    df.loc[(df.name == u'Marie Sk\u0142odowska-Curie') &\
      (df.year == 1911), 'country'] = 'France'
    df = df[~((df.name=='Sidney Altman') & (df.year==1990))] # selects all rows 'not' (~) Sidney Altma 1990
    # the missing categories are mostly people whose category name wasn't in standard format
    # fix that:
    df.loc[df.name=='Alexis Carrel', 'category'] = 'Physiology or Medicine'
    df.loc[df.name=='Ilya Ilyich Mechnikov', 'category'] = 'Physiology or Medicine'
    df.loc[df.name=='Amartya Sen', 'category'] = 'Economics'
    df.drop(df.loc[df.name.str.startswith('Leopold Ru') & (df.year != 1939)].index, inplace=True)
    df.loc[df.name.str.startswith(u'R\xf3bert B\xe1r\xe1ny'), 'category'] = 'Physiology or Medicine'
    # drop null gender entries (they are all institutions/organizations)
    df = df[df.gender.notnull()] # remove genderless entries
    return df

df = clean_data(df)

## Dealing with Times and Dates

In [52]:
df.loc[:20, ['name', 'date_of_birth']]

,name,date_of_birth
0,George de Hevesy,1885-08-01 00:00:00
1,Albert Szent-Györgyi,1893-09-16 00:00:00
2,Kofi Annan,1938-04-08
3,Odysseas Elytis,1911-11-02
4,Giorgos Seferis,1900-03-13
5,Rigoberta Menchú,1959-01-09
6,Richard Adolf Zsigmondy,1865-04-01 00:00:00
8,Róbert Bárány,1876-04-22 00:00:00
10,Philipp Lenard,1862-06-07 00:00:00
12,Fritz Haber,1868-12-09 00:00:00


In [53]:
pd.to_datetime(df.date_of_birth, errors='raise') # no errors
pd.to_datetime(df.date_of_death, errors='raise')


0      1966-07-05
1      1986-10-22
2             NaT
3      1996-03-18
4      1971-09-20
5             NaT
6      1929-09-23
8      1936-04-08
10     1947-05-20
12     1934-01-29
13     1947-10-04
14     1942-08-03
15     1960-04-24
16     1946-06-06
17     1928-08-30
18     1931-02-26
19     1927-07-05
20     1914-04-02
21     1918-04-20
22     1938-05-04
23     1941-09-09
24     1976-02-01
25     1970-08-01
26     1940-04-26
27     1949-03-30
28     1945-03-31
29     1955-08-12
30     1959-06-09
31     1941-03-04
32     1957-08-05
          ...    
988    1995-11-04
989    2016-09-28
991    2013-08-30
992    1988-01-15
994    1989-12-22
995    1995-06-25
996    1997-09-05
997    1970-11-21
998    1941-08-07
999           NaT
1000          NaT
1001          NaT
1002          NaT
1004          NaT
1005          NaT
1007          NaT
1009          NaT
1010          NaT
1011          NaT
1012          NaT
1013   1998-02-08
1014          NaT
1015          NaT
1016   1974-06-09
1017      

In [54]:
# if there was an error raised, this is one way to know which row caused it:
for i, row in df.iterrows():
    try:
        pd.to_datetime(row.date_of_death, errors='raise')
    except:
        print('{}({}, {})'.format(row.date_of_death.ljust(30), row['name'], i))

#
with_death_dates = df[df.date_of_death.notnull()]
bad_dates = pd.isnull(pd.to_datetime(with_death_dates.date_of_death, errors='coerce'))
with_death_dates[bad_dates][['category', 'date_of_death', 'name']]

,category,date_of_death,name


___
# FINAL
# COPY OF GETTING THE DIRTY DATA:


In [1]:
import numpy as np
import pandas as pd

from pymongo import MongoClient

def get_mongo_database(db_name, host='localhost', port=27017, username=None, password=None):
    '''Get (or create) named database from MongoDB with/out authentication'''
    if username and password:
        mongo_uri = 'mongodb://{}:{}@{}/{}'.format(username, password, host, db_name)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)
    return conn[db_name]

def mongo_to_dataframe(db_name, collection_name, query={}, host='localhost',
                       port=27017, username=None, password=None, no_id=True):
    '''Create a Pandas DataFrame from MongoDB collection'''
    db = get_mongo_database(db_name, host, port, username, password)
    cursor = db[collection_name].find(query)
    df = pd.DataFrame(list(cursor))
    if no_id:
        del df['_id']
    return df

def dataframe_to_mongo(dframe, db_name, collection_name, host='localhost',
                 port=27017, username=None, password=None):
    '''save a dataframe to mongodb collection'''
    db = get_mongo_database(db_name, host, port, username, password)
    records = dframe.to_dict('records')  # 'records' puts it into our list-of-dicts format
    db[collection_name].insert_many(records)

def delete_collection(db_name, collection_name, host='localhost',
                 port=27017, username=None, password=None):
    '''save a dataframe to mongodb collection'''
    db = get_mongo_database(db_name, host, port, username, password)
    db[collection_name].delete_many({}) # empty filter deletes all entries


DB_NOBEL_PRIZE = 'nobel_prize' # use string constants or a spell error in retrieval will create new table.
COLL_WINNERS = 'winners' # winners collection

#----------------------------
# From json file.
#----------------------------

with open('data/nwinners.json') as f:
    df = pd.read_json(f)

# Make sure mongodb is clear (so we don't duplicate data), then
# save to Mongo for next section
db = get_mongo_database(DB_NOBEL_PRIZE)
db[COLL_WINNERS].delete_many({})  # deletes everything (no filter)
dataframe_to_mongo(df, DB_NOBEL_PRIZE, COLL_WINNERS) # save to Mongo for next section

#----------------------------
# From mongodb collection
#----------------------------

#df = mongo_to_dataframe(DB_NOBEL_PRIZE, COLL_WINNERS)
df.count()


born_in           1069
category          1069
country           1069
date_of_birth     1060
date_of_death      712
gender            1060
link              1069
name              1069
place_of_birth    1060
place_of_death     712
text              1069
year              1069
dtype: int64

# The final clean_data() function

In [2]:
def clean_data(df):
    df.name = df.name.str.replace('*', '') # Take out the asterisks in people's name
    df.name = df.name.str.strip() # Strip out ending whitespace
    df = df.replace('', np.nan)
    df_born_in = df[df.born_in.notnull()]   # save off the entries with born_in
    df = df[df.born_in.isnull()] # eliminates the rows with 'born_in' values (they are dups)
    df = df.drop('born_in', axis=1) # drop this column so no overlap in upcoming join()
    df.drop(df[df.year==1809].index, inplace=True)
    df = df[~(df['name']=='Marie Curie')]
    df.loc[(df.name == u'Marie Sk\u0142odowska-Curie') &\
      (df.year == 1911), 'country'] = 'France'
    df = df[~((df.name=='Sidney Altman') & (df.year==1990))] # selects all rows 'not' (~) Sidney Altma 1990
    # drop duplicate awards where the recipient had 2 countries.  Randomize which
    # country is dropped by first randomizing the index
    df = df.reindex(np.random.permutation(df.index))
    df = df.drop_duplicates(['name', 'year']) # if duplicate on these 2 fields, drop after 1st entry
    df = df.sort_index() #restores the index so no longer random
    # similarly drop entries where two born_ins are given (DAY this fixed problem
    # where doing a join to get both country and born_in was generating duplicates
    # for some rows and therefore causing the index size to mismatch later)
    df_born_in = df_born_in.reindex(np.random.permutation(df_born_in.index))
    df_born_in = df_born_in.drop_duplicates(['name', 'year']) # if duplicate on these 2 fields, drop after 1st entry
    df_born_in = df_born_in.sort_index() #restores the index so no longer random
    # DAY added section here to copy born_in info in the main df
    match_cols = ['name', 'year']
    df = df.join(df_born_in.set_index(match_cols).loc[:,'born_in'], how='left', on=match_cols)
    df_born_in = df_born_in.drop('country', axis=1)
    df_born_in = df_born_in.join(df.set_index(match_cols).loc[:,'country'], how='left', on=match_cols)
    # end DAY section
    # the missing categories are mostly people whose category name wasn't in standard format
    # fix that:
    df.loc[df.name=='Alexis Carrel', 'category'] = 'Physiology or Medicine'
    df.loc[df.name=='Ilya Ilyich Mechnikov', 'category'] = 'Physiology or Medicine'
    df.loc[df.name=='Amartya Sen', 'category'] = 'Economics'
    df.drop(df.loc[df.name.str.startswith('Leopold Ru') & (df.year != 1939)].index, inplace=True)
    df.loc[df.name.str.startswith(u'R\xf3bert B\xe1r\xe1ny'), 'category'] = 'Physiology or Medicine'
    # drop null gender entries (they are all institutions/organizations)
    df = df[df.gender.notnull()] # remove genderless entries
    df.date_of_birth = pd.to_datetime(df.date_of_birth)
    df.date_of_death = pd.to_datetime(df.date_of_death, errors='coerce')
    df['award_age'] = df.year - pd.DatetimeIndex(df.date_of_birth).year
    return df, df_born_in
    
df_clean, df_born_in = clean_data(df)

df_clean.count()

category          874
country           874
date_of_birth     874
date_of_death     592
gender            874
link              874
name              874
place_of_birth    874
place_of_death    592
text              874
year              874
born_in            88
award_age         874
dtype: int64

In [6]:
df_clean.head()



,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year,born_in,award_age
3,Literature,Austria,1946-10-20,NaT,female,https://en.wikipedia.org/wiki/Elfriede_Jelinek,Elfriede Jelinek,Mürzzuschlag,NaN,"Elfriede Jelinek , Literature, 2004",2004,NaN,58
4,Chemistry,Summary,1906-09-06,1987-12-02,male,https://en.wikipedia.org/wiki/Luis_Federico_Le...,Luis Federico Leloir,Paris,Catamarca Province,"Luis Federico Leloir , Chemistry, 1970",1970,France,64
5,Physiology or Medicine,Summary,1887-04-10 00:00:00,1971-09-21,male,https://en.wikipedia.org/wiki/Bernardo_Houssay,Bernardo Houssay,Buenos Aires,Buenos Aires,"Bernardo Houssay , Physiology or Medicine, 1947",1947,NaN,60
8,Peace,Argentina,1931-11-26,NaT,male,https://en.wikipedia.org/wiki/Adolfo_P%C3%A9re...,Adolfo Pérez Esquivel,Buenos Aires,NaN,"Adolfo Pérez Esquivel , Peace, 1980",1980,NaN,49
11,Peace,Argentina,1878-11-01 00:00:00,1959-05-05,male,https://en.wikipedia.org/wiki/Carlos_Saavedra_...,Carlos Saavedra Lamas,Buenos Aires,Buenos Aires,"Carlos Saavedra Lamas , Peace, 1936",1936,NaN,58


In [4]:
df_born_in.country.unique()




array([nan, u'United States', u'United Kingdom', u'Poland', u'Israel',
       u'Belgium', u'Canada', u'Australia', u'Denmark', u'Netherlands',
       u'France', u'East Timor', u'Germany', u'Ukraine', u'Sweden',
       u'Switzerland', u'Ireland', u'Summary', u'Austria'], dtype=object)

# Save the data

In [5]:
if True:
    db = get_mongo_database(DB_NOBEL_PRIZE)
    db[COLL_WINNERS].delete_many({})  # deletes everything (no filter) to make sure we're starting fresh
    db['winners_born_in'].delete_many({})  # deletes everything (no filter) to make sure we're starting fresh
    dataframe_to_mongo(df_clean, DB_NOBEL_PRIZE, COLL_WINNERS) # save to Mongo for next section
    dataframe_to_mongo(df_born_in, DB_NOBEL_PRIZE, 'winners_born_in')

with open('data/nwinners_clean.json', 'w') as f:
    df_clean.to_json(f, date_format='iso', orient='records')
with open('data/nwinners_born_in.json', 'w') as f:
    df_born_in.to_json(f, date_format='iso', orient='records')
 

# also save a local copy on sqlite
if False:
    import sqlalchemy

    engine = sqlalchemy.create_engine('sqlite:///data/nobel_prize.db')
    df_clean.to_sql('winners', engine)


ValueError: NaTType does not support utcoffset

# Merge MiniBio Dataframe

In [313]:
with open('data/minibios.json') as f:
    df_winners_bios = pd.read_json(f)
    
df_winners_all = pd.merge(df_clean, df_winners_bios, how='outer', on='link')
df_winners_all.count()

category          1029
country           1029
date_of_birth     1029
date_of_death      691
gender            1029
link              1081
name              1029
place_of_birth    1029
place_of_death     691
text              1029
year              1029
born_in            181
award_age         1029
bio_image         1020
image_urls        1081
mini_bio          1081
dtype: int64

In [314]:
# remove redundancies
# remove duplicates of any rows that share a 'link' and 'year' field after removing
# any rows without a name field
df_winners_all = df_winners_all[~df_winners_all.name.isnull()].drop_duplicates(subset=['link', 'year'])
df_winners_all.count()
df_winners_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 873 entries, 0 to 1028
Data columns (total 16 columns):
category          873 non-null object
country           873 non-null object
date_of_birth     873 non-null datetime64[ns]
date_of_death     592 non-null datetime64[ns]
gender            873 non-null object
link              873 non-null object
name              873 non-null object
place_of_birth    873 non-null object
place_of_death    592 non-null object
text              873 non-null object
year              873 non-null float64
born_in           88 non-null object
award_age         873 non-null float64
bio_image         824 non-null object
image_urls        873 non-null object
mini_bio          873 non-null object
dtypes: datetime64[ns](2), float64(2), object(12)
memory usage: 115.9+ KB


In [315]:
db['winners_all'].delete_many({})  # deletes everything (no filter) to make sure we're starting fresh
dataframe_to_mongo(df_winners_all, DB_NOBEL_PRIZE, 'winners_all')



ValueError: NaTType does not support utcoffset

In [316]:
df_winners_all.columns

Index([      u'category',        u'country',  u'date_of_birth',
        u'date_of_death',         u'gender',           u'link',
                 u'name', u'place_of_birth', u'place_of_death',
                 u'text',           u'year',        u'born_in',
            u'award_age',      u'bio_image',     u'image_urls',
             u'mini_bio'],
      dtype='object')

In [317]:
# Save json file (need to deal with datetimes which don't json)

# here is one way I could have done it.
import datetime
print(df_winners_all.date_of_birth.apply(lambda x: x.isoformat()))

#but I am using Pandas' built-in to_json which deals with datetime
with open('data/nwinners_all.json', 'w') as f:
    df_winners_all.to_json(f, date_format='iso', orient='records')
    
    
# ARCHIVE
if False:
    import datetime
    from dateutil import parser
    import json

    class JSONDateTimeEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, (datetime.date, datetime.datetime)):
                return obj.isoformat()
            else:
                return json.JSONEncoder.default(self, obj)

    def mydumps(obj):
        return json.dumps(obj, cls=JSONDateTimeEncoder)
    now_str = mydumps({'time': datetime.datetime.now()})
    print(now_str)

    from datetime import datetime
    import dateutil

    d = datetime.datetime.now()
    d_iso = d.isoformat()  # convert to text
    print(d_iso)

    d = dateutil.parser.parse(d_iso) # back to a datetime object
    print(d)




0       1946-10-20T00:00:00
1       1887-04-10T00:00:00
3       1927-10-08T00:00:00
6       1906-09-06T00:00:00
9       1878-11-01T00:00:00
11      1967-02-24T00:00:00
12      1951-09-30T00:00:00
13      1936-03-11T00:00:00
14      1936-10-10T00:00:00
15      1882-08-26T00:00:00
16      1887-07-22T00:00:00
17      1918-10-04T00:00:00
18      1901-03-27T00:00:00
19      1862-06-05T00:00:00
20      1833-02-19T00:00:00
21      1843-05-21T00:00:00
22      1905-09-30T00:00:00
23      1892-09-06T00:00:00
24      1886-09-13T00:00:00
25      1906-06-19T00:00:00
27      1941-02-19T00:00:00
28      1947-06-08T00:00:00
29      1927-05-22T00:00:00
31      1920-05-29T00:00:00
32      1928-06-13T00:00:00
33      1919-04-22T00:00:00
34      1924-08-23T00:00:00
35      1940-05-24T00:00:00
37      1932-06-18T00:00:00
38      1936-11-19T00:00:00
               ...         
990     1918-07-15T00:00:00
991     1932-04-26T00:00:00
993     1929-11-02T00:00:00
994     1939-05-07T00:00:00
996     1932-11-06T0

In [318]:
df = df_winners_all